In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [122]:
#Quantum Monte Carlo - Diffusion

class dmc:
    
    def __init__(self, num_walkers_start=1000, target_walkers=1000, walkers_start_position =0.0, num_steps = 1000, time_step = 0.1, e_r= 0.,
                 burn_in = 300, num_bins = 200, x_min = -20, x_max = 20):
        #Initialization
        self.num_walkers_start =  num_walkers_start
        self.walkers_start_position = walkers_start_position
        self.target_walkers = target_walkers
        self.num_steps = num_steps
        self.time_step = time_step
        self.e_r = e_r
        self.burn_in = burn_in
        self.num_bins = num_bins
        self.x_min = x_min
        self.x_max = x_max
        
        self.walkers = np.full(num_walkers_start, walkers_start_position)
        

    def potential_harmonic(self):
        """Computes the potential values at given locations x for a simple harmonic oscillator
        Input:  m     :: the mass of the walkers
                omega :: the angular frequency of the walkers
        """      
        v = 0.5*self.walkers*2
        return v

    def potential_morse(x):
        """Computes the potential values at given locations x for a simple harmonic oscillator
        Input:  m     :: the mass of the walkers
                omega :: the angular frequency of the walkers
                a     :: defining parameter - will be set to 1 for ease of solution
        """
        v = 0.5*(np.exp( -2*a*x ) - 2*np.exp( -a*x ) )
        return v

    def run_dmc(self):
        """Simulates a quantum system using the quantum diffusion method"""
        wave_function = np.zeros(self.num_bins)
        e_0 = []

        for i in range(self.num_steps):
            #Perform a random walk
            for j in range(self.walkers.size):
                self.walkers[j] += np.random.normal() * np.sqrt(self.time_step)
            num_walkers_old = self.walkers.size
            
            #Determine the weight function
            energy = np.zeros_like(self.walkers)                                       #finding the weight for each walker
            energy = 1.0 -(self.potential_harmonic() - self.e_r) * self.time_step
            weight = np.int_(energy)                                              #convert to int per the algorithm
            new_walkers = np.zeros_like(self.walkers)                                  #how many new walkers each walker spawns
            new_walkers = np.minimum(3, weight)

            #Make a new array of walkers
            keep = np.compress(new_walkers > 0, self.walkers)        #get an array of each walker still existing
            duplicate1 = np.compress(new_walkers > 1, self.walkers)  #get an array of each walker duplicated
            duplicate2 = np.compress(new_walkers > 2, self.walkers)  #get an array of each walker twice duplicated
            self.walkers = np.append(keep, duplicate1)                  #combine the above arrays into a complete array of walkers
            self.walkers = np.append(self.walkers, duplicate2)
            
            #Once burn-in is complete, add values to the histogram
            if i>self.burn_in:
                hist, bin_edges = np.histogram(self.walkers, bins=self.num_bins, range=(self.x_min, self.x_max))
                wave_function += hist                                             #add the walker distribution at this step to the total distribution
                e_0.append(np.mean(energy))                                       #E_0 estimate at this step

            #Adjust values for next iteration
            e_r = np.mean(energy) + (1. - (self.walkers.size - self.target_walkers) / (num_walkers_old * self.time_step))        #NEED TO CHECK UNITS
            
            print(i, e_r, self.walkers.size)
            
        return wave_function, bin_edges, e_0


In [123]:
dmc_test_1 = dmc()
wave, bins, e0 = dmc_test_1.run_dmc()
plt.hist(wave, bins=bins)

trial = np.linspace(0, len(e0), len(e0))
plt.figure(2)
plt.plot(trial, e0)

0 7.03029607977 497
1 14.240518974 393
2 19.0128655616 333
3 23.5530233911 284
4 28.0525210554 262
5 31.3103251944 234
6 35.376712793 221
7 37.8361442868 210
8 39.9538279708 205
9 41.3188573605 196
10 43.7397139405 184
11 46.6826192964 180
12 48.0113740948 174
13 50.0053616633 167
14 52.2008711949 164
15 53.3653743487 160
16 54.7749675955 158
17 55.7018937432 154
18 57.4837576488 148
19 60.0007221201 144
20 61.754056292 142
21 62.9458313814 137
22 65.4620329369 133
23 67.74124351 128
24 70.3095656926 128
25 70.6983922076 123
26 73.7359793976 120
27 75.7749710013 117
28 77.6690673935 117
29 78.0917802213 112
30 81.6725929026 110
31 83.30383453 108
32 84.8101253585 108
33 84.9932647532 106
34 86.8427824645 103
35 89.4084022477 102
36 90.3644615227 101
37 91.4335715745 99
38 93.2432042937 99
39 93.2402256914 99
40 93.2423019472 99
41 93.2490132132 99
42 93.6536263783 95
43 97.7241552097 93
44 99.9997272215 91
45 102.148048292 91
46 102.256206631 90
47 103.478836366 89
48 104.612407094 89


KeyboardInterrupt: 

In [93]:
a = np.array([0, 5, 10, 15, 20, 25])
b = np.compress(a>10, a)
print(b)

[15 20 25]
